In [1]:
import tensorflow as tf
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.resnet import decode_predictions
from keras.layers import Input,UpSampling2D,Flatten,BatchNormalization,Dense,Dropout,GlobalAveragePooling2D
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from PIL import Image
import PIL

In [2]:
NUM_CLASSES = 3
NUM_EPOCHS = 100

In [3]:
IMAGE_SIZE = [224, 224]
train = '/content/drive/MyDrive/data/train'
test = '/content/drive/MyDrive/data/test'

In [4]:
folders = glob('/content/drive/MyDrive/data/train/*')
print(len(folders))

3


In [5]:
IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

In [6]:
LOSS_METRICS = ['accuracy']

In [7]:
BATCH_SIZE_TRAINING = 50
BATCH_SIZE_VALIDATION = 50

In [8]:
resnet=ResNet50(include_top=False, weights='imagenet',input_shape=IMAGE_SIZE + [3])

94773248/94765736 [==============================] - 1s 0us/step


In [9]:
from keras.models import Model,Sequential

In [10]:
model = Sequential()
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

In [11]:
model.layers[0].trainable = False

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 6,147
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
from keras import optimizers

In [14]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

In [15]:
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [16]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    preprocessing_function=preprocess_input)


test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = data_generator.flow_from_directory(
        train,
        target_size=(224, 224),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
        test,
        target_size=(224, 224),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical')

Found 360 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [18]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [19]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

checkpoint = ModelCheckpoint(filepath='Image_classification_resnet50.h5', 
                               verbose=1, save_best_only=True)


callbacks = [checkpoint, lr_reducer]

In [20]:
history = model.fit(
        train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
        callbacks=callbacks)

Epoch 1/100
8/8 [==============================] - ETA: 0s - loss: 3.8620 - accuracy: 0.3417 
Epoch 00001: val_loss improved from inf to 6.62997, saving model to Image_classification_resnet50.h5
8/8 [==============================] - 412s 51s/step - loss: 3.8620 - accuracy: 0.3417 - val_loss: 6.6300 - val_accuracy: 0.3333
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 4.1605 - accuracy: 0.3333
Epoch 00002: val_loss improved from 6.62997 to 5.07983, saving model to Image_classification_resnet50.h5
8/8 [==============================] - 4s 550ms/step - loss: 4.1605 - accuracy: 0.3333 - val_loss: 5.0798 - val_accuracy: 0.3333
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 4.4557 - accuracy: 0.3056
Epoch 00003: val_loss improved from 5.07983 to 3.93924, saving model to Image_classification_resnet50.h5
8/8 [==============================] - 5s 623ms/step - loss: 4.4557 - accuracy: 0.3056 - val_loss: 3.9392 - val_accuracy: 0.3333
Epoch 4/100
8/8 [=====

In [21]:
model.save('human_activity_recognitation.h5')

In [33]:
print(train_generator.class_indices)

{'YOGA  - tree pose': 0, 'YOGA - goddess pose': 1, 'walking': 2}


In [35]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/images.jpg', target_size = (224, 224,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
img_class = (model.predict_classes(test_image)>0.6).astype("int32")
prediction = img_class[0]
classname = img_class[0]
for k, v in train_generator.class_indices.items():
    if v == classname:
        print(k)

YOGA - goddess pose


In [38]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/images (1).jpg', target_size = (224, 224,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
img_class = (model.predict_classes(test_image)>0.6).astype("int32")
prediction = img_class[0]
classname = img_class[0]
print(img_class, prediction, classname)
for k, v in train_generator.class_indices.items():
    if v == classname:
        print(k)

[0] 0 0
YOGA  - tree pose


In [22]:
from tensorflow import keras
m = keras.models.load_model('human_activity_recognitation.h5')